In [1]:
import requests
import pandas as pd
import json
import sqlite3
import xlwt 
from xlwt import Workbook
import pymongo

In [2]:
def insert(user_id, recommend):
    if(count(user_id) > 0):
        for r in recommend:
            update(user_id, r)
    else:
        d={'user_id': user_id, 'recommend': recommend}
        return col.insert_one(d)
def count(user_id):
    return col.count_documents({"user_id": user_id})
def update(user_id, recommend):
    query = { "user_id": user_id}
    newvalues = { "$push": { "recommend": recommend } }
    return col.update_one(query, newvalues)
def search(user_id):
    query = { "user_id": user_id}
    return col.find(query)
def recommend(user_id):
    for x in search(user_id):
        return x['recommend']
def delete(user_id):
    query = { "user_id": user_id}
    return col.delete_many(query)

In [5]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
col = mydb["recommend"]
df = pd.read_csv("../test_watch_log.csv")
c=0

wb = Workbook()
sheet = wb.add_sheet('video_recommend') 
sheet.write(0,0,'user_id')
sheet.write(0,1,'video_id')
sheet.write(0,2,'videos recommemed')
sheet.write(0,3,'video is in recommend list')
for index, row in df.iterrows():
    rst = False
    try:
        recommend_num = 50
        result = requests.get('http://localhost:5000/video_id_to_video?video_id=' \
                                  + str(row['video_id']) +'&nums=' + str(recommend_num))
        r = json.loads(result.text)
        rmd = r['videos_id']
        id = int(row['user_id'])
        insert(id,rmd)
        
        recommend_list = recommend(id)
        if row['video_id'] in recommend_list:
            c+=1
            rst = True
        sheet.write(index+1,0,str(row['user_id']))
        sheet.write(index+1,1,str(row['video_id']))
        sheet.write(index+1,2,str(recommend_list))
        sheet.write(index+1,3,str(rst))
    except:
        continue
    
wb.save('result.xls')
print(c)

3197


In [15]:
cursor = col.find({})
for document in cursor:
    print(document)

In [4]:
col.remove()

<ipython-input-4-096d42e75f2b>:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  col.remove()


{'n': 3278, 'ok': 1.0}

In [9]:
print(recommend(415827))

None


In [16]:
insert(123,['123'])